# Поиск и сравнение текста

In [1]:
import os
import difflib  # модуль для обнаружения различий

Предположим, что в слове, состоящем из последовательности символов, произошла ошибка. Например, телефонный номер '89034706554' был неправильно распознан как '39034706334'  Как оценить степень ошибки?

In [2]:
s0='89034706554'   # эталон
s='39034706334'
s==s0

False

Строки не равны между собой. Если они равны по длине, то мы можем сравнить их посимвольно.

In [3]:
assert( len(s0)==len(s) )  # убедимся в равенстве длины

n=0 # сюда будем складывать количество одинаковых символов
for i in range(len(s)):
    if s0[i]==s[i]:
        n+=1
        
n, n/len(s)
        

(8, 0.7272727272727273)

8 из 11 символов оказались одинаковыми. В данном случае мы можем утверждать, что качество распознавания данного номера составило примерно 73%. 

Однако, что делать, если количество символов в результате ошибки изменилось, или в слове произошло сразу несколько ошибок???

In [4]:
s0='"ТЕРМИНАТОР"'
s='Терминатор-2'

s==s0,  len(s)==len(s0)

(False, True)

Строки различаются, но одинаковой длины. При этом что-то подсказывает, что между ними есть некое сходство. Попробуем достичь единообразия, сменив регистр.

In [5]:
s1=s0.lower()
s2=s.lower()
s1, s2

('"терминатор"', 'терминатор-2')

Даже после этого посимвольное сравнение даст 0 одинаковых символов. Воспользуемся инструментом из библиотеки difflib.

In [6]:
p=difflib.SequenceMatcher(None, s1, s2).ratio()
p

0.8333333333333334

In [7]:
p=difflib.SequenceMatcher(None, s0, s).ratio()
print('{} ~ {} = {:.3}'.format(s0,s,p))

"ТЕРМИНАТОР" ~ Терминатор-2 = 0.0833


Между исходными словами сходство всего лишь в одной букве.

In [8]:
# определим функцию, которая будет сравнивать два слова
def strcompare(a,b):
    sm=difflib.SequenceMatcher(None, a, b)
    
    #m=sm.find_longest_match(0,11,0,11)
    #print('Наиболее длинная последовательность в словах {} и {}:'.format(a,b))
    
    for block in sm.get_matching_blocks():
        if block[2]:
            print("совпадение из {2} символов в позициях a[{0}] и b[{1}]".format(*block))
    print()
    print('Чтобы получить из {} => {}:'.format(a,b))
    for opcode in sm.get_opcodes():
        print("    {:7s} a[{}:{}] b[{}:{}]".format(*opcode))

    p=sm.ratio()
    print('Сходство {:.1f}%'.format(p*100))
    
    return p

In [9]:
strcompare(s1,s2)

совпадение из 10 символов в позициях a[1] и b[0]

Чтобы получить из "терминатор" => терминатор-2:
    delete  a[0:1] b[0:0]
    equal   a[1:11] b[0:10]
    replace a[11:12] b[10:12]
Сходство 83.3%


0.8333333333333334

In [10]:
p=strcompare('Волоколамск','молокозавод')

совпадение из 5 символов в позициях a[1] и b[1]
совпадение из 1 символов в позициях a[7] и b[7]

Чтобы получить из Волоколамск => молокозавод:
    replace a[0:1] b[0:1]
    equal   a[1:6] b[1:6]
    replace a[6:7] b[6:7]
    equal   a[7:8] b[7:8]
    replace a[8:11] b[8:11]
Сходство 54.5%


In [11]:
p=strcompare('стройное тело','дойная телка')

совпадение из 3 символов в позициях a[3] и b[1]
совпадение из 4 символов в позициях a[8] и b[6]

Чтобы получить из стройное тело => дойная телка:
    replace a[0:3] b[0:1]
    equal   a[3:6] b[1:4]
    replace a[6:8] b[4:6]
    equal   a[8:12] b[6:10]
    replace a[12:13] b[10:12]
Сходство 56.0%


### О "расстоянии" между двумя строками

Многие знают игру, как из МУХА сделать СЛОН. За один шаг разрешается менять только одну букву. Эта игра демонстрирует подход для измерения различий между любыми последовательностями символов.

Расстояние между словами измеряется количеством операций редактирования, которые нужны, чтобы из одного слова получить другое. При этом разные варианты таких метрик могут считать прибавление или удаление нескольких символов за одну операцию, или игнорировать пробелы, или не учитывать регистр букв и т.п. Более продвинутые системы распознавания слов учитывают падежные окончания, контекст использования, синонимы и пр. Такие системы используются для исправления запросов в поисковых системах (напр. Яндекс).

Сходные явления в биологии:
- Операция редактирования ~ мутация
- Расстояние между строками ~ возраст общего предка
- Приведение к одному регистру ~ трансляция ДНК в РНК
- Не учитывать окончания (стемминг) ~ вырожденность генетического кода
- Контекст использования ~ гидрофильный/гидрофобный конец, отрицательно/положительно заряженный 
